In [3]:
from openai import AzureOpenAI
import openpyxl
from openpyxl import Workbook
import os

# Set your OpenAI API key
# openai.api_key = 'your-openai-api-key'

# Create or load the Excel workbook
file_path = 'gpt4_api_calls.xlsx'
if os.path.exists(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
else:
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(['Call Number', 'Prompt', 'Output Text', 'Model', 'Temperature', 'Max Tokens', 'Top P', 'Frequency Penalty', 'Presence Penalty'])

In [4]:
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

In [5]:
# Function to log API call details to the Excel file
def log_to_excel(call_number, model, call_id, messages, prompt, response, temperature, seed, max_tokens, usage, filter):
    row = [
        call_number,
        model,
        call_id,
        messages,
        prompt,
        response,
        temperature,
        seed,
        max_tokens,
        usage,
        filter
    ]
    sheet.append(row)
    workbook.save(file_path)


In [6]:

# Function to call the OpenAI API and log the response
def call_gpt4(model, messages, temperature, seed, max_tokens):
    call_number = sheet.max_row  # Call number is the current row count
    response = client.chat.completions.create(
        model = model,
        seed = seed,
        messages = messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    log_to_excel(call_number, model, response.id, str(messages), messages[-1]["content"], str(response.choices[0].message.content), temperature, seed, max_tokens, str(response.usage), str(response.prompt_filter_results))
    return response